In [9]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from pymongo import MongoClient
from dotenv import load_dotenv

In [10]:
# Nama database dan collection
db_name = 'youtube_rewind_indonesia'
collection_label = 'labeling_2015'
collection_name = 'full_labeling_2015'

# Memuat value dari file .env
load_dotenv()

mongodb_url = os.getenv('URL_SANDY')
local_url = os.getenv('URL_LOCAL')

#### Mengambil data dari MongoDB ke dalam dataframe

In [11]:
# Membuat koneksi ke MongoDB
client = MongoClient(local_url)
db = client[db_name]  # Ganti dengan nama database Anda
labeling_2015_collection = db[collection_label]
full_labeling_2015_collection = db[collection_name]

# Mengambil data dari koleksi MongoDB
labeled_data = list(labeling_2015_collection.find())
unlabeled_data = list(full_labeling_2015_collection.find())

# Konversi data menjadi dataframe
df_labeled = pd.DataFrame(labeled_data)
df_unlabeled = pd.DataFrame(unlabeled_data)

# Memisahkan fitur dan label dari data yang berlabel
X_labeled = df_labeled['textOriginal']
y_labeled = df_labeled['label']

# Menggunakan TfidfVectorizer untuk mengubah teks menjadi fitur
vectorizer = TfidfVectorizer()
X_labeled_tfidf = vectorizer.fit_transform(X_labeled)

# Melatih model menggunakan Naive Bayes
model = MultinomialNB()
model.fit(X_labeled_tfidf, y_labeled)

# Memproses data yang tidak berlabel
X_unlabeled = df_unlabeled['textOriginal']
X_unlabeled_tfidf = vectorizer.transform(X_unlabeled)

# Memprediksi label untuk data yang tidak berlabel
y_unlabeled_pred = model.predict(X_unlabeled_tfidf)

# Menambahkan prediksi ke dalam dataframe data tidak berlabel
df_unlabeled['predicted_label'] = y_unlabeled_pred

# Mengupdate koleksi MongoDB dengan prediksi label
for index, row in df_unlabeled.iterrows():
    full_labeling_2015_collection.update_one(
        {"_id": row["_id"]},
        {"$set": {"predicted_label": row["predicted_label"]}}
    )

print("Prediksi label untuk data tidak berlabel telah selesai dan diupdate dalam koleksi MongoDB.")

Prediksi label untuk data tidak berlabel telah selesai dan diupdate dalam koleksi MongoDB.


In [12]:
df_positif = df_unlabeled[df_unlabeled['predicted_label'] == 'positif']
df_negatif = df_unlabeled[df_unlabeled['predicted_label'] == 'negatif']
df_netral = df_unlabeled[df_unlabeled['predicted_label'] == 'netral']

In [13]:
df_positif

,_id,textOriginal,predicted_label
1,6651837620ee1df904ba499b,rewind ter the best sih nice,positif
4,6651837620ee1df904ba499e,anjir keren gilak youtubers indonesia,positif
10,6651837620ee1df904ba49a4,wow seru banget video kompak,positif
11,6651837620ee1df904ba49a5,am only one who always watching this video al ...,positif
12,6651837620ee1df904ba49a6,wow kerensnjshshsjsh aku suka banget,positif
...,...,...,...
9934,6651837620ee1df904ba7068,keren bangat tetap karya youtubers indonesia,positif
9950,6651837620ee1df904ba7078,musicnya the best,positif
9984,6651837620ee1df904ba709a,suka pesan moral kena asap sempat landa wilaya...,positif
9988,6651837620ee1df904ba709e,keren abis,positif


In [14]:
df_negatif

,_id,textOriginal,predicted_label
3,6651837620ee1df904ba499d,andai,negatif
37,6651837620ee1df904ba49bf,di sni seblum game moba entod,negatif
50,6651837620ee1df904ba49cc,kok ga seru tapi gp lanjut,negatif
88,6651837620ee1df904ba49f2,jangan bikin tahun nanti isi artis semua,negatif
90,6651837620ee1df904ba49f4,yutub rewind gada bang yon mantap jiwa jadi ci...,negatif
...,...,...,...
9738,6651837620ee1df904ba6fa4,ko edho mana ko edho,negatif
9809,6651837620ee1df904ba6feb,cuma give away ga jelas,negatif
9913,6651837620ee1df904ba7053,kok kak edhozel ga yah kecewa deh,negatif
9925,6651837620ee1df904ba705f,ko edho mana,negatif


In [15]:
df_netral

,_id,textOriginal,predicted_label
0,6651837620ee1df904ba499a,judul lagu menit apa si ka,netral
2,6651837620ee1df904ba499c,lagu nya apa aja yang tau,netral
5,6651837620ee1df904ba499f,harus charlie charlie pencil,netral
6,6651837620ee1df904ba49a0,sini azka jadi bocah tk sekarang udah besar bgt,netral
7,6651837620ee1df904ba49a1,lagu detik,netral
...,...,...,...
9995,6651837620ee1df904ba70a5,rachel godard mana,netral
9996,6651837620ee1df904ba70a6,kangen banget gua,netral
9997,6651837620ee1df904ba70a7,kak chandra kak laurentius beatbox nya keren b...,netral
9998,6651837620ee1df904ba70a8,we mis lord rando,netral


#### Accuracy Test (soon)

In [16]:
# Jika data tidak berlabel memiliki label asli untuk keperluan evaluasi
if 'label' in df_unlabeled.columns:
    print("\nEvaluasi kinerja model:")
    print("Accuracy Score:", accuracy_score(df_unlabeled['label'], df_unlabeled['predicted_label']))
    print("Classification Report:\n", classification_report(df_unlabeled['label'], df_unlabeled['predicted_label']))
else:
    print("Data tidak berlabel tidak memiliki label asli untuk evaluasi.")

Data tidak berlabel tidak memiliki label asli untuk evaluasi.
